# Prompt Chaining and Sequencing Tutorial

## Overview

This tutorial explores the concepts of prompt chaining and sequencing in the context of working with large language models. We'll use OpenAI's GPT models and the LangChain library to demonstrate how to connect multiple prompts and build logical flows for more complex AI-driven tasks.

## Motivation

As AI applications become more sophisticated, there's often a need to break down complex tasks into smaller, manageable steps. Prompt chaining and sequencing allow us to guide language models through a series of interrelated prompts, enabling more structured and controlled outputs. This approach is particularly useful for tasks that require multiple stages of processing or decision-making.

## Key Components

1. **Basic Prompt Chaining**: Connecting the output of one prompt to the input of another.
2. **Sequential Prompting**: Creating a logical flow of prompts to guide the AI through a multi-step process.
3. **Dynamic Prompt Generation**: Using the output of one prompt to dynamically generate the next prompt.
4. **Error Handling and Validation**: Implementing checks and balances within the prompt chain.

## Method Details

We'll start by setting up our environment with the necessary libraries. Then, we'll explore basic prompt chaining by connecting two simple prompts. We'll move on to more complex sequential prompting, where we'll guide the AI through a multi-step analysis process. Next, we'll demonstrate how to dynamically generate prompts based on previous outputs. Finally, we'll implement error handling and validation techniques to make our prompt chains more robust.

Throughout the tutorial, we'll use practical examples to illustrate these concepts, such as a multi-step text analysis task and a dynamic question-answering system.

## Conclusion

By the end of this tutorial, you'll have a solid understanding of how to implement prompt chaining and sequencing in your AI applications. These techniques will enable you to tackle more complex tasks, improve the coherence and relevance of AI-generated content, and create more interactive and dynamic AI-driven experiences.

## Setup

Let's start by importing the necessary libraries and setting up our environment.

In [3]:
import os
import re

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

## Basic Prompt Chaining

Let's start with a simple example of prompt chaining. We'll create two prompts: one to generate a short story, and another to summarize it.

In [2]:
# Define prompt templates
story_prompt = PromptTemplate(
    input_variables=["genre"],
    template="Write a short {genre} story in 3-4 sentences."
)

summary_prompt = PromptTemplate(
    input_variables=["story"],
    template="Summarize the following story in one sentence:\n{story}"
)

# Chain the prompts
def story_chain(genre):
    """Generate a story and its summary based on a given genre.

    Args:
        genre (str): The genre of the story to generate.

    Returns:
        tuple: A tuple containing the generated story and its summary.
    """
    story = (story_prompt | llm).invoke({"genre": genre}).content
    summary = (summary_prompt | llm).invoke({"story": story}).content
    return story, summary

# Test the chain
genre = "science fiction"
story, summary = story_chain(genre)
print(f"Story: {story}\n\nSummary: {summary}")

Story: In the year 2147, humanity discovered a way to communicate with their future selves through time-locked messages sent via quantum entanglement. When Ava received a cryptic warning from her future self, she struggled to decipher its meaning: "Trust the shadow, not the light." As a solar flare threatened to wipe out Earth's power grid, she realized the warning was about a hidden faction that thrived in the chaos. Embracing the darkness, Ava united the underground resistance, ensuring that humanity would not just survive, but rise anew from the ashes.

Summary: In 2147, Ava deciphers a cryptic warning from her future self about a hidden faction amidst a solar flare crisis, leading her to unite an underground resistance that helps humanity not only survive but thrive in the chaos.


## Sequential Prompting

Now, let's create a more complex sequence of prompts for a multi-step analysis task. We'll analyze a given text for its main theme, tone, and key takeaways.

In [4]:
# Define prompt templates for each step
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="Identify the main theme of the following text:\n{text}"
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="Describe the overall tone of the following text:\n{text}"
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="Given the following text with the main theme '{theme}' and tone '{tone}', what are the key takeaways?\n{text}"
)

def analyze_text(text):
    """Perform a multi-step analysis of a given text.

    Args:
        text (str): The text to analyze.

    Returns:
        dict: A dictionary containing the theme, tone, and key takeaways of the text.
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content
    return {"theme": theme, "tone": tone, "takeaways": takeaways}

# Test the sequential prompting
sample_text = "The rapid advancement of artificial intelligence has sparked both excitement and concern among experts. While AI promises to revolutionize industries and improve our daily lives, it also raises ethical questions about privacy, job displacement, and the potential for misuse. As we stand on the brink of this technological revolution, it's crucial that we approach AI development with caution and foresight, ensuring that its benefits are maximized while its risks are minimized."

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key.capitalize()}: {value}\n")

Theme: The main theme of the text is the duality of artificial intelligence advancements, highlighting both the potential benefits and ethical concerns associated with its development. It emphasizes the need for cautious and responsible approaches to harness AI's advantages while addressing issues like privacy, job displacement, and potential misuse.

Tone: The overall tone of the text is cautious and balanced. It expresses a sense of excitement about the potential benefits of artificial intelligence, while simultaneously acknowledging the concerns and ethical dilemmas it presents. The emphasis on the need for careful consideration and foresight reflects a responsible and thoughtful approach to the development of AI, highlighting both optimism and wariness.

Takeaways: Here are the key takeaways based on the provided theme and tone:

1. **Duality of AI Advancements**: The text highlights the dual nature of artificial intelligence, presenting both significant benefits and serious ethica

## Dynamic Prompt Generation

In this section, we'll create a dynamic question-answering system that generates follow-up questions based on previous answers.

In [5]:
# Define prompt templates
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely:\n{question}"
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Based on the question '{question}' and the answer '{answer}', generate a relevant follow-up question."
)

def dynamic_qa(initial_question, num_follow_ups=3):
    """Conduct a dynamic Q&A session with follow-up questions.

    Args:
        initial_question (str): The initial question to start the Q&A session.
        num_follow_ups (int): The number of follow-up questions to generate.

    Returns:
        list: A list of dictionaries containing questions and answers.
    """
    qa_chain = []
    current_question = initial_question

    for _ in range(num_follow_ups + 1):  # +1 for the initial question
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})
        
        if _ < num_follow_ups:  # Generate follow-up for all but the last iteration
            current_question = (follow_up_prompt | llm).invoke({"question": current_question, "answer": answer}).content

    return qa_chain

# Test the dynamic Q&A system
initial_question = "What are the potential applications of quantum computing?"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"Q{i+1}: {qa['question']}")
    print(f"A{i+1}: {qa['answer']}\n")

Q1: What are the potential applications of quantum computing?
A1: Potential applications of quantum computing include:

1. **Cryptography**: Breaking classical encryption methods and developing quantum-secure communication.
2. **Optimization**: Solving complex optimization problems in logistics, finance, and supply chain management.
3. **Drug Discovery**: Simulating molecular interactions for faster pharmaceutical development.
4. **Material Science**: Designing new materials with specific properties at the quantum level.
5. **Artificial Intelligence**: Enhancing machine learning algorithms and data analysis.
6. **Financial Modeling**: Improving risk assessment and portfolio optimization.
7. **Weather Forecasting**: Enhancing predictive models for climate and weather patterns.
8. **Quantum Simulation**: Studying complex quantum systems in physics and chemistry. 

These applications leverage quantum superposition and entanglement to perform calculations beyond the capability of classical

## Error Handling and Validation

In this final section, we'll implement error handling and validation in our prompt chains to make them more robust.

In [8]:
# Define prompt templates
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a 4-digit number related to the topic: {topic}. Respond with ONLY the number, no additional text."
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="Is the number {number} truly related to the topic '{topic}'? Answer with 'Yes' or 'No' and explain why."
)

def extract_number(text):
    """Extract a 4-digit number from the given text.

    Args:
        text (str): The text to extract the number from.

    Returns:
        str or None: The extracted 4-digit number, or None if no valid number is found.
    """
    match = re.search(r'\b\d{4}\b', text)
    return match.group() if match else None

def robust_number_generation(topic, max_attempts=3):
    """Generate a topic-related number with validation and error handling.

    Args:
        topic (str): The topic to generate a number for.
        max_attempts (int): Maximum number of generation attempts.

    Returns:
        str: A validated 4-digit number or an error message.
    """
    for attempt in range(max_attempts):
        try:
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)
            
            if not number:
                raise ValueError(f"Failed to extract a 4-digit number from the response: {response}")
            
            # Validate the relevance
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("yes"):
                return number
            else:
                print(f"Attempt {attempt + 1}: Number {number} was not validated. Reason: {validation}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
    
    return "Failed to generate a valid number after multiple attempts."

# Test the robust number generation
topic = "World War II"
result = robust_number_generation(topic)
print(f"Final result for topic '{topic}': {result}")

Final result for topic 'World War II': 1945
